<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [5]:
spacex_df.head(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
5,7,2013-12-03,22:41:00,F9 v1.1,CCAFS LC-40,SES-8,3170.0,GTO,SES,No attempt,0,28.562302,-80.577356
6,8,2014-01-06,22:06:00,F9 v1.1,CCAFS LC-40,Thaicom 6,3325.0,GTO,Thaicom,No attempt,0,28.562302,-80.577356
7,9,2014-04-18,19:25:00,F9 v1.1,CCAFS LC-40,SpaceX CRS-3,2296.0,LEO (ISS),NASA (CRS),Controlled (ocean),0,28.562302,-80.577356
8,10,2014-07-14,15:15:00,F9 v1.1,CCAFS LC-40,OG2 Mission 1 6 Orbcomm-OG2 satellites,1316.0,LEO,Orbcomm,Controlled (ocean),0,28.562302,-80.577356
9,11,2014-08-05,8:00:00,F9 v1.1,CCAFS LC-40,AsiaSat 8,4535.0,GTO,AsiaSat,No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Initial the map
CCAFS_LC_40_cor = [28.562302,-80.577356]
CCAFS_SLC_40_cor = [28.563197,-80.576820]
KSC_LC_39A_cor = [28.573255,-80.646895]
VAFB_SLC_4E_cor = [34.632834,-120.610745]
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
circle = folium.Circle(CCAFS_LC_40_cor, radius=10, color='#d35400', fill=True).add_child(folium.Popup('CCAFS LC-40 Launch Center'))
circle2 = folium.Circle(CCAFS_SLC_40_cor, radius=10, color='#d35400', fill=True).add_child(folium.Popup('CCAFS SLC-40 Launch Center'))
circle3 = folium.Circle(KSC_LC_39A_cor, radius=10, color='#d35400', fill=True).add_child(folium.Popup('KSC LC-39A Launch Center'))
circle4 = folium.Circle(VAFB_SLC_4E_cor, radius=10, color='#d35400', fill=True).add_child(folium.Popup('VAFB SLC-4E Launch Center'))
marker = folium.map.Marker(
    CCAFS_LC_40_cor,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS LC-40 LaunchSite',
        )
    )
marker2 = folium.map.Marker(
    CCAFS_SLC_40_cor,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS SLC-40 LaunchSite',
        )
    )
marker3 = folium.map.Marker(
    KSC_LC_39A_cor,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'KSC LC-39A LaunchSite',
        )
    )
marker4 = folium.map.Marker(
    VAFB_SLC_4E_cor,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'VAFB SLC-4E LaunchSite',
        )
    )
site_map.add_child(circle)
site_map.add_child(circle2)
site_map.add_child(circle3)
site_map.add_child(circle4)
site_map.add_child(marker)
site_map.add_child(marker2)
site_map.add_child(marker3)
site_map.add_child(marker4)

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
# Task 2: Mark the success/failed launches for each site on the map


In [10]:
s = spacex_df.groupby("Launch Site")["class"].value_counts()

In [11]:
s

Launch Site   class
CCAFS LC-40   0        19
              1         7
CCAFS SLC-40  0         4
              1         3
KSC LC-39A    1        10
              0         3
VAFB SLC-4E   0         6
              1         4
Name: class, dtype: int64

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [12]:
spacex_df.rename(columns = {"class":"Class"},inplace=True)

In [13]:
spacex_df.head()

,Launch Site,Lat,Long,Class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [156]:
#marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [157]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
#site_map = folium.Map(location = nasa_coordinate, zoom_start = 4)

# instantiate a mark cluster object for the incidents in the dataframe
launch_outcomes = plugins.MarkerCluster().add_to(site_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(spacex_df.Lat, spacex_df.Long, spacex_df.Class):
    if label == 0:
        folium.Marker(
            location=[lat, lng],
            icon=folium.Icon(color='red'),
            popup=label,
            #marker_color = 'green'
        ).add_to(launch_outcomes)
        continue
    elif label == 1:
        folium.Marker(
            location=[lat, lng],
            icon=folium.Icon(color='green'),
            popup=label,
            #marker_color = 'green'
        ).add_to(launch_outcomes)
        continue
    #print(label)
# display map'''
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [21]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
prox_cor = pd.DataFrame({"Launch Site" : ["LC","SLC","KSC","VAFB"],
                       "Lat" : [28.56249,28.56333,28.5794,34.63307],
                        "Long" : [-80.56794,-80.56807,-80.57287,-120.62636]})

'''for lat,long,lat2,long2, in zip(launch_sites_df.Lat,launch_sites_df.Long):
    distance_coastline = calculate_distance(lat, long , coastline_lat, coastline_lon)'''

'for lat,long,lat2,long2, in zip(launch_sites_df.Lat,launch_sites_df.Long):\n    distance_coastline = calculate_distance(lat, long , coastline_lat, coastline_lon)'

In [22]:
prox_cor

,Launch Site,Lat,Long
0,LC,28.56249,-80.56794
1,SLC,28.56333,-80.56807
2,KSC,28.57940,-80.57287
3,VAFB,34.63307,-120.62636


In [23]:
distance = []

In [24]:
for lat,long,lat2,long2, in zip(launch_sites_df.Lat,launch_sites_df.Long,prox_cor.Lat,prox_cor.Long):
    distance_coastline = calculate_distance(lat, long , lat2, long2)
    distance.append(distance_coastline)
    print(distance_coastline)

0.9201602416868537
0.8549357511988961
7.263019862308647
1.429295095735873


In [25]:
distance

[0.9201602416868537, 0.8549357511988961, 7.263019862308647, 1.429295095735873]

In [26]:
prox_cor["Distance"] = distance

In [27]:
prox_cor

,Launch Site,Lat,Long,Distance
0,LC,28.56249,-80.56794,0.920160
1,SLC,28.56333,-80.56807,0.854936
2,KSC,28.57940,-80.57287,7.263020
3,VAFB,34.63307,-120.62636,1.429295


In [28]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
for lat,long,dist,lat2,long2 in zip(prox_cor.Lat,prox_cor.Long,prox_cor.Distance,launch_sites_df.Lat,launch_sites_df.Long):
    #print(lat,long,dist)
    distance_marker = folium.map.Marker(
        [lat,long],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(dist),
            )
        )
    lines=folium.PolyLine(locations=[[lat,long],[lat2,long2]], weight=1)
    site_map.add_child(lines)
    site_map.add_child(distance_marker)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [17]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
prox_cor_road = pd.DataFrame({"Launch Site" : ["LC","SLC","KSC","VAFB"],
                       "Lat" : [28.56231,28.56328,28.57345,34.66403],
                        "Long" : [-80.57062,-80.57079,-80.65548,-120.45685]})
distance = []
for lat,long,lat2,long2, in zip(launch_sites_df.Lat,launch_sites_df.Long,prox_cor_road.Lat,prox_cor_road.Long):
    distance_coastline = calculate_distance(lat, long , lat2, long2)
    distance.append(distance_coastline)
    #print(distance_coastline)
prox_cor_road["Distance"] = distance
for lat,long,dist,lat2,long2 in zip(prox_cor_road.Lat,prox_cor_road.Long,prox_cor_road.Distance,launch_sites_df.Lat,launch_sites_df.Long):
    #print(lat,long,dist)
    distance_marker = folium.map.Marker(
        [lat,long],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(dist),
            )
        )
    lines=folium.PolyLine(locations=[[lat,long],[lat2,long2]], weight=1)
    site_map.add_child(lines)
    site_map.add_child(distance_marker)
site_map

In [18]:
prox_cor_rail = pd.DataFrame({"Launch Site" : ["LC","SLC","KSC","VAFB"],
                       "Lat" : [28.56376,28.56464,28.57349,34.63706],
                        "Long" : [-80.58688,-80.58667,-80.65415,-120.62357]})
distance = []
for lat,long,lat2,long2, in zip(launch_sites_df.Lat,launch_sites_df.Long,prox_cor_rail.Lat,prox_cor_rail.Long):
    distance_coastline = calculate_distance(lat, long , lat2, long2)
    distance.append(distance_coastline)
    #print(distance_coastline)
prox_cor_rail["Distance"] = distance
for lat,long,dist,lat2,long2 in zip(prox_cor_rail.Lat,prox_cor_rail.Long,prox_cor_rail.Distance,launch_sites_df.Lat,launch_sites_df.Long):
    #print(lat,long,dist)
    distance_marker = folium.map.Marker(
        [lat,long],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(dist),
            )
        )
    lines=folium.PolyLine(locations=[[lat,long],[lat2,long2]], weight=1)
    site_map.add_child(lines)
    site_map.add_child(distance_marker)
site_map

In [19]:
prox_cor_city = pd.DataFrame({"Launch Site" : ["LC","SLC","KSC","VAFB"],
                       "Lat" : [28.58738,28.58738,28.58738,34.65137],
                        "Long" : [-80.65132,-80.65132,-80.65132,-120.45753]})
distance = []
for lat,long,lat2,long2, in zip(launch_sites_df.Lat,launch_sites_df.Long,prox_cor_city.Lat,prox_cor_city.Long):
    distance_coastline = calculate_distance(lat, long , lat2, long2)
    distance.append(distance_coastline)
    #print(distance_coastline)
prox_cor_city["Distance"] = distance
for lat,long,dist,lat2,long2 in zip(prox_cor_city.Lat,prox_cor_city.Long,prox_cor_city.Distance,launch_sites_df.Lat,launch_sites_df.Long):
    #print(lat,long,dist)
    distance_marker = folium.map.Marker(
        [lat,long],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(dist),
            )
        )
    lines=folium.PolyLine(locations=[[lat,long],[lat2,long2]], weight=1)
    site_map.add_child(lines)
    site_map.add_child(distance_marker)
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
